In [1]:
# impoerting necessaRY LIBRARIES
import os
import sys
import math
import numpy as np
from pyspark.mllib.recommendation import ALS
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.mllib.feature import Normalizer
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.mllib.feature import HashingTF, IDF


In [2]:
def get_movie_docbyid(movieid,read_tags_1m_data):
  return read_tags_1m_data.filter(lambda x: x[0] == movieid).map(lambda x :  [i  for i in x[1]])

In [3]:
def doc2vec(movieid,data,features_length,hashingTF,idf):
  new_movie = get_movie_docbyid(movieid,data)
  #print(new_movie.take(1))
  candidateTf = hashingTF.transform(new_movie)
  candidateTfIdf = idf.transform(candidateTf)
  return candidateTfIdf

In [4]:
def cosineSimilarity(candidateTfIdf,frequencyDenseVectors_0):
  frequencyDenseVectors_1 =  candidateTfIdf.map(lambda vector: DenseVector(vector.toArray()))
  y1 =   frequencyDenseVectors_1.collect()
  re = frequencyDenseVectors_0.map(lambda x : (x.dot(y1[0]))/(x.norm(2)*y1[0].norm(2)))
  return re

In [5]:
def get_topn_similarities(n,re,a):
  result1=re.collect()
  
  dict1 = {}
  list_index  = []
  for i in result1:
    if (i > 0.1)&(i<0.999999) :
      #print(result1.index(i))
      dict1[a[result1.index(i)]] = i
      #list_index.append(result1.index(i))
  sorted_d = sorted(dict1.items(), key=lambda x: x[1])
  top5_r  = sorted_d[-n:]
  for i in top5_r:
    list_index.append(i[0])
  print(list_index)
  return top5_r

In [6]:
def topn_df(top5_r,schema):
  top5_rec = sc.parallelize(top5_r)
  df_top5 = spark.createDataFrame([[str(top5_r1[0]), float(top5_r1[1])] for top5_r1 in top5_r ],('movienum','similarity_score'),schema)
  return df_top5

In [7]:
def cont_rec_view(tag_movies,df_top5):
  comdf_top5 = tag_movies.join(df_top5, df_top5['movienum'] == tag_movies.movieid_num )
#comdf_top5.na.drop()
#comdf_top5.show(1572)
#comdf_top5 = tags.filter( df_top5['_1'] == tags['_c1'] ).show()
  comdf_top5.show()

In [8]:
def feutuervec(features_length,documents):
  hashingTF = HashingTF(features_length)
  tf = hashingTF.transform(documents)


  tf.cache()
  idf = IDF().fit(tf)
  tfidf = idf.transform(tf)
  return tfidf,hashingTF,idf

In [9]:
def content_recomm(data_rdd,feature_len,frequencyDenseVectors_0,a,hashingTF,idf):
   # Case for user had viewed multiple movies
  #260,1,16,25,32,335,379,296,858,50
  new_user_multiple = [260,1,16,25,32,296,858,50]

  #top5_df or dataframe 
  schema = StructType([   
                              StructField("movienum", StringType(), True)
                              ,StructField("similarity_score", FloatType(), True)
                          ])

  similar_movies_df = spark.createDataFrame([], schema)

  for movieid in new_user_multiple:
    print(movieid)
    features = doc2vec(movieid,data_rdd,feature_len,hashingTF,idf)
    #print(features.take(1))
    cos_smi_fe = cosineSimilarity(features,frequencyDenseVectors_0)
    #print(cos_smi_fe)
    topnsim = get_topn_similarities(5,cos_smi_fe,a)
    #topnsim.show()
    #print(topnsim)
    similar_movie_df = topn_df(topnsim,schema)
    similar_movies_df = similar_movies_df.union(similar_movie_df)

  cont_recm = similar_movies_df.orderBy("similarity_score", ascending = False).limit(10)
  return cont_recm
  #find rating send if rating ggreater than 3:


In [10]:
def show_df_combined(read_tags,mov):
  tag_movies = read_tags.join(mov, read_tags.movieid_num == mov.movieId).drop('movieId')
#df_index = df.select("*").withColumn("id", monotonically_increasing_id())
  tag_movies_id = tag_movies.select("*").withColumn("id", monotonically_increasing_id())
  #tag_movies_id.show()
  return tag_movies

In [11]:
def Read_data_files(ratings_path, movies_path):
  read_ratings_1m = sc.textFile(ratings_path)
  read_ratings_1m_header = read_ratings_1m.take(1)[0]
  read_ratings_1m_data = read_ratings_1m.filter(lambda line : line != read_ratings_1m_header).map(lambda line:line.split(",")).map(lambda tokens:   (int(tokens[0]),int(tokens[1]),float(tokens[2])))
  read_movies_1m = sc.textFile(movies_path)
  read_movies_1m_header = read_movies_1m.take(1)[0]
  read_movies_1m_data = read_movies_1m.filter(lambda line : line != read_movies_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[0]),tokens[1]))
  
  return read_ratings_1m_data, read_movies_1m_data
  
  

In [12]:
def Train_ALS_model(train,test):
  seed = 5
  iterations = 10
  reg_parameter = 0.1
  rank = 8
  error = 0
  
  cf_model = ALS.train(train, rank, seed = seed, iterations = iterations, lambda_=reg_parameter)
  test_predict = test.map(lambda tuple_element:(tuple_element[0],tuple_element[1]))
  prediction = cf_model.predictAll(test_predict).map(lambda r:((r[0], r[1]), r[2]))
  #joing this predicted ratings with original values for finding the accuracy of the model
  r_p = test.map(lambda x : ((int(x[0]), int(x[1])), float(x[2]))).join(prediction)
  error1 = math.sqrt(r_p.map(lambda r : (r[1][0] - r[1][1])**2).mean())
  
  return error1,cf_model
  
  

In [13]:
def get_newuser_recommendations(new_u_ratings,clf_new_model,read_movies_1m_data,read_ratings_1m_data,user_id = 0):
  new_ur_movie_ids = map(lambda x: x[1], new_u_ratings)
  new_user_unrated_movies_RDD = read_movies_1m_data.filter(lambda x: x[0] not in new_ur_movie_ids).map(lambda x: (user_id, x[0]))
  new_user_recommendations_RDD = clf_new_model.predictAll(new_user_unrated_movies_RDD)
  
  print(new_user_recommendations_RDD.take(5))
  pred_ratings = new_user_recommendations_RDD.map(lambda x : (x[1],x[2])).sortByKey()
  
  ratings = read_ratings_1m_data.map(lambda x : (x[1],x[2])).groupByKey()
  print(ratings.take(2))
  movie_avg_ratings = ratings.map(lambda x: (x[0], len(x[1]), sum([i for i in x[1]])/len(x[1])))
  
  print(movie_avg_ratings.take(2))
  
  predicted_r_movie_titles = pred_ratings.join(read_movies_1m_data).join(movie_avg_ratings)
  
  print(predicted_r_movie_titles.take(5))
  
  flating_predictions = predicted_r_movie_titles.map(lambda x : (x[1][0][1],x[1][0][0],x[1][1]))
  
  result = flating_predictions.filter(lambda x : x[2] > 25).takeOrdered(25, key=lambda x: -x[1])
  
  return flating_predictions
  
  

In [14]:
def Colloborativefiltering(ratings_path,movies_path):
  read_ratings_1m_data, read_movies_1m_data =  Read_data_files(ratings_path, movies_path)
  read_ratings_1m_data.cache()
  train, test = read_ratings_1m_data.randomSplit([7, 3], seed=0)
  user_id = 0
  #error1,model = Train_ALS_model(train,test)
  seed = 5
  iterations = 10
  reg_parameter = 0.1
  rank = 8
  error = 0
  
  #cf_model = ALS.train(train, rank, seed = seed, iterations = iterations, lambda_=reg_parameter)
  #test_predict = test.map(lambda tuple_element:(tuple_element[0],tuple_element[1]))
  #prediction = cf_model.predictAll(test_predict).map(lambda r:((r[0], r[1]), r[2]))
  #joing this predicted ratings with original values for finding the accuracy of the model
  #r_p = test.map(lambda x : ((int(x[0]), int(x[1])), float(x[2]))).join(prediction)
  #error1 = math.sqrt(r_p.map(lambda r : (r[1][0] - r[1][1])**2).mean())
  
  
  
  
  
  
  
  
  Added_new_ur_data,new_u_ratings  = Add_new_user(read_ratings_1m_data)
  
  
  
  clf_new_model = ALS.train(Added_new_ur_data, 8, seed = 5, iterations = 10, lambda_=0.1)
  
  
  new_ur_movie_ids = map(lambda x: x[1], new_u_ratings)
  new_user_unrated_movies_RDD = read_movies_1m_data.filter(lambda x: x[0] not in new_ur_movie_ids).map(lambda x: (user_id, x[0]))
  new_user_recommendations_RDD = clf_new_model.predictAll(new_user_unrated_movies_RDD)
  
  #print(new_user_recommendations_RDD.take(5))
  pred_ratings = new_user_recommendations_RDD.map(lambda x : (x[1],x[2])).sortByKey()
  print(pred_ratings.take(3))
  ratings = read_ratings_1m_data.map(lambda x : (x[1],x[2])).groupByKey()
  df1 = spark.createDataFrame(ratings, ('movieid', 'ratings'))
  a = [i.ratings[0] for i in df1.select('ratings').collect()]
  id = [i.movieid for i in df1.select('movieid').collect()] 
  s = []
  for i,j in enumerate(a):
    s.append((id[i], len(j), np.sum(j)/len(j)))

  c_rdd = sc.parallelize(s)
  print(c_rdd.take(3))
  c_rdd1 = c_rdd.map(lambda x: (x[0], x[1]))
  predicted_r_movie_titles = pred_ratings.join(read_movies_1m_data).join(c_rdd1)
  
  print(predicted_r_movie_titles.take(5))
  
  flating_predictions = predicted_r_movie_titles.map(lambda x : (x[0],x[1][0][1],x[1][0][0],x[1][1]))
  
  result = flating_predictions.filter(lambda x : x[3] > 25).takeOrdered(25, key=lambda x: -x[2])
  
  
  #result1 = get_newuser_recommendations(new_u_ratings,clf_new_model,read_movies_1m_data,read_ratings_1m_data)
  return result
  
  
  

In [15]:
def Add_new_user(read_ratings_1m_data):
  # Adding new user to the datset
  user_id = 0

  # creading userid, movieid, ratings RDD for a new user (movie Ids are choosen randomly based on his interest)

  new_u_ratings = [
       (0,260,4), # Star Wars (1977)
       (0,1,3), # Toy Story (1995)
       (0,16,3), # Casino (1995)
       (0,25,4), # Leaving Las Vegas (1995)
       (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
       (0,335,1), # Flintstones, The (1994)
       (0,379,1), # Timecop (1994)
       (0,296,3), # Pulp Fiction (1994)
       (0,858,5) , # Godfather, The (1972)
       (0,50,4) # Usual Suspects, The (1995)
      ]

  new_u_ratings_RDD = sc.parallelize(new_u_ratings)
  new_u_ratings_RDD.take(10)

  # Adding this user data to the original user-ratings data

  Added_new_ur_data =  read_ratings_1m_data.union(new_u_ratings_RDD)
  return Added_new_ur_data, new_u_ratings

In [16]:
def ensemble(con_r,cbf,wr,wcbf,mov):
  newcr = con_r.limit(wr)
  newcr.show()
  cbf1 = sc.parallelize(cbf[:wcbf])
  newcbf = spark.createDataFrame(cbf1,('movie_num','moviename','rat','count'))
  newcbf.show()
  re = mov.join(newcr, mov.movieId == newcr.movienum).drop('similarity_score')
  re2 = mov.join(newcbf, mov.movieId == newcbf.movie_num).drop('rat').drop('count').drop('moviename')
  re.show()
  re2.show()
  result = re.union(re2)
  result.show()
  display(result)
  
 
  
  
  
  

In [17]:
# Here the main function starts  
if __name__ == "__main__": 
  movies_path = "/FileStore/tables/movies.csv"
  ratings_path = "/FileStore/tables/ratings.csv"
  
  read_tags_1m = sc.textFile("/FileStore/tables/tags_l.csv")
  mov = spark.read.csv('/FileStore/tables/movies.csv', header=True)
  read_tags_1m_header = read_tags_1m.take(1)[0]
  read_tags_1m_data = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[1]),tokens[2]))\
  .groupByKey().sortByKey()
  #print(read_tags_1m_data.take(3))
  documents = read_tags_1m_data.map(lambda x :   [i  for i in x[1]])
  #print(documents.take(2))
  read_tags = spark.createDataFrame(read_tags_1m_data,('movieid_num', 'document'))
  a = [i.movieid_num for i in read_tags.select('movieid_num').distinct().collect()]
 
  bagfwords = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:((tokens[2]), int(1)))\
  .reduceByKey(lambda x,y:x+y)
  features_length = bagfwords.count()
  a.sort()
  tfidf,hashingTF,idf = feutuervec(features_length,documents)
  #print(tfidf.take(2))
  frequencyDenseVectors_0 = tfidf.map(lambda vector: DenseVector(vector.toArray()))
  #print(frequencyDenseVectors_0.take(5))
  cr_out = content_recomm(read_tags_1m_data,features_length,frequencyDenseVectors_0,a,hashingTF,idf)
  cr_out.show()
  tag_movies = show_df_combined(read_tags,mov)
  cont_rec_view(tag_movies,cr_out)
  
  result = Colloborativefiltering(ratings_path,movies_path)
  #print(error1)
  print(result)
  #print(result.collect())
  
  
  
  
  
  # ALS Model Trainig and Colloborative filtering
  
  


260
[33493, 5378, 98695, 1210, 1196]
1
[4886, 6377, 78499, 2355, 3114]
16
[8042, 89069, 85190, 120795, 1213]
25
[2505, 4144, 7759, 31785, 2933]
32
[4878, 135859, 157563, 96610, 114935]
296
[18, 6874, 7438, 1729, 1089]
858
[2023, 1466, 151591, 4262, 1221]
50
[1625, 47, 1617, 2762, 97973]
+--------+------------------+
movienum| similarity_score|
+--------+------------------+
 1213|0.7567710590313329|
 1089|0.7559349517270009|
 3114|0.7522343935343732|
 1221|0.7208180882462243|
 2355|0.6634401708197005|
 1729|0.6631345862127239|
 78499|0.6574993605321294|
 6377|0.6474311477254641|
 7438|0.6292780262334008|
 6874|0.6208954589449202|
+--------+------------------+

+-----------+--------------------+--------------------+--------------------+--------+------------------+
movieid_num| document| title| genres|movienum| similarity_score|
+-----------+--------------------+--------------------+--------------------+--------+------------------+
 1089|[[great dialogue,...|Reservoir Dogs (1...|Crime|Mystery|Thr...| 1089|0.7559349517270009|
 1213|[[crime, dark com...| Goodfellas (1990)| Crime|Drama| 1213|0.7567710590313329|
 1221|[[boring, Mafia, ...|Godfather: Part I...| Crime|Drama| 1221|0.7208180882462243|
 1729|[[dark comedy, Qu...| Jackie Brown (1997)|Crime|Drama|Thriller| 1729|0.6631345862127239|
 2355|[[animation, Disn...|Bug's Life, A (1998)|Adventure|Animati...| 2355|0.6634401708197005|
 3114|[[Pixar, sequel b...| Toy Story 2 (1999)|Adventure|Animati...| 3114|0.7522343935343732|
 6377|[[Pixar, funny, P...| Finding Nemo (2003)|Adventure|Animati...| 6377|0.6474311477254641|
 6874|[[Kick-Butt Women...|Kill Bill: Vol. 1...|Action|Crime|Thri...| 6874|0.6208954589449202|
 7438|[[martial arts, m...|Kill Bill: Vol. 2...|Action|Drama|Thri...| 7438|0.6292780262334008|
 78499|[[boring, overrat...| Toy Story 3 (2010)|Adventure|Animati...| 78499|0.6574993605321294|
+-----------+--------------------+--------------------+--------------------+--------+------------------+

[(2, 1.9381774091699322), (3, 1.9328798489940882), (4, 1.2678377614282974)]
[(1296, 7470, 3.8941767068273094), (2352, 7517, 3.720167620061195), (828, 1736, 3.1474654377880182)]
[(60, ((1.8825780367337985, '"Indian in the Cupboard'), 7601)), (120, ((0.9854814114517367, 'Race the Sun (1996)'), 168)), (180, ((2.5896070495123125, 'Mallrats (1995)'), 9727)), (240, ((1.5698335097328275, 'Hideaway (1995)'), 867)), (300, ((3.041502028796079, 'Quiz Show (1994)'), 21807))]
[(109370, 'Magic & Bird: A Courtship of Rivals (2010)', 4.205458047054151, 27), (4171, "Long Night's Journey Into Day (2000)", 4.034671592120159, 35), (858, '"Godfather', 3.9401285962269315, 60904), (139086, 'Small Potatoes - Who Killed the USFL? (2009)', 3.9250156017837905, 26), (171495, 'Cosmos', 3.9183881373494582, 157), (79677, '"Two Escobars', 3.9133231390548304, 78), (1221, '"Godfather: Part II', 3.8746254985856208, 38875), (173351, 'Wow! A Talking Fish! (1983)', 3.8728742513819707, 47), (108043, 'Milius (2013)', 3.8611850190449717, 44), (160317, 'Roots (2016)', 3.860586669993372, 28), (116002, 'History of the Eagles (2013)', 3.8520613377611097, 32), (91762, '"Last Lions', 3.8439770709231045, 38), (70186, 'Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)', 3.8351099378886944, 35), (296, 'Pulp Fiction (1994)', 3.801915117453738, 92406), (100553, 'Frozen Planet (2011)', 3.793612933978338, 402), (182615, 'The Landlord (2007)', 3.7694492637625405, 26), (165239, 'Supersonic (2016)', 3.7613698898885604, 51), (128981, 'Music for One Apartment and Six Drummers (2001)', 3.758114602698226, 31), (69483, '"Not on Your Life (Verdugo', 3.7437483443350423, 28), (104069, 'Louis C.K.: Oh My God (2013)', 3.7311551931029308, 918), (136447, 'George Carlin: You Are All Diseased (1999)', 3.7299635638346347, 163), (95973, '"Classe américaine', 3.727265394410656, 38), (171011, 'Planet Earth II (2016)', 3.7197041565270705, 853), (170705, 'Band of Brothers (2001)', 3.719112688606158, 984), (318, '"Shawshank Redemption', 3.

In [18]:
# Ensembiling the results with intermidiate steps
ensemble(cr_out,result,5,5,mov)

movieId,title,genres,movienum
1089,Reservoir Dogs (1992),Crime|Mystery|Thriller,1089
1213,Goodfellas (1990),Crime|Drama,1213
1221,"Godfather: Part II, The (1974)",Crime|Drama,1221
2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,2355
3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,3114
109370,Magic & Bird: A Courtship of Rivals (2010),Documentary,109370
4171,Long Night's Journey Into Day (2000),Documentary,4171
858,"Godfather, The (1972)",Crime|Drama,858
139086,Small Potatoes - Who Killed the USFL? (2009),Documentary,139086
171495,Cosmos,(no genres listed),171495


In [19]:
result

Out[19]: [(109370, 'Magic & Bird: A Courtship of Rivals (2010)', 4.205458047054151, 27),
 (4171, "Long Night's Journey Into Day (2000)", 4.034671592120159, 35),
 (858, '"Godfather', 3.9401285962269315, 60904),
 (139086,
 'Small Potatoes - Who Killed the USFL? (2009)',
 3.9250156017837905,
 26),
 (171495, 'Cosmos', 3.9183881373494582, 157),
 (79677, '"Two Escobars', 3.9133231390548304, 78),
 (1221, '"Godfather: Part II', 3.8746254985856208, 38875),
 (173351, 'Wow! A Talking Fish! (1983)', 3.8728742513819707, 47),
 (108043, 'Milius (2013)', 3.8611850190449717, 44),
 (160317, 'Roots (2016)', 3.860586669993372, 28),
 (116002, 'History of the Eagles (2013)', 3.8520613377611097, 32),
 (91762, '"Last Lions', 3.8439770709231045, 38),
 (70186,
 'Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)',
 3.8351099378886944,
 35),
 (296, 'Pulp Fiction (1994)', 3.801915117453738, 92406),
 (100553, 'Frozen Planet (2011)', 3.793612933978338, 402),
 (182615, 'The Landlord (2007)', 3.7694492637625405, 26),
 (165239, 'Supersonic (2016)', 3.7613698898885604, 51),
 (128981,
 'Music for One Apartment and Six Drummers (2001)',
 3.758114602698226,
 31),
 (69483, '"Not on Your Life (Verdugo', 3.7437483443350423, 28),
 (104069, 'Louis C.K.: Oh My God (2013)', 3.7311551931029308, 918),
 (136447,
 'George Carlin: You Are All Diseased (1999)',
 3.7299635638346347,
 163),
 (95973, '"Classe américaine', 3.727265394410656, 38),
 (171011, 'Planet Earth II (2016)', 3.7197041565270705, 853),
 (170705, 'Band of Brothers (2001)', 3.719112688606158, 984),
 (318, '"Shawshank Redemption', 3.716193365655629, 97999)]